<a href="https://colab.research.google.com/github/Dkepffl/Causal-Inference/blob/main/Code/Merge_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **데이터와 라이브러리 불러오기**

In [ ]:
## load libraries
import pandas as pd

In [ ]:
## load dataframe

### 주요 변수
BoxOffice = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/Boxoffice.csv')

OTT_sales = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/OTT.csv')
OTT_UserRate = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/User.csv')

### 그 외 변수
Covid = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/Covid.csv')
N_Covid = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/Covid19.csv')

GDP = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/GDP.csv')
Unemployed = pd.read_csv('/content/drive/MyDrive/Causal_Inference/Data/Unemployed.csv')

## **데이터 병합**

In [ ]:
# 빈 데이터프레임 생성
df = pd.DataFrame()

In [ ]:
# BoxOffice 데이터셋에서 Year, Month, 관객수, 매출액, 스크린수, 개봉 영화관 수
df['Year']=BoxOffice['Year']
df['Month']=BoxOffice['Month']
df['Num_Audience']=BoxOffice['Total_audience_count']
df['Movie_Revenue']=BoxOffice['Total_revenue']
df['Num_Theaters']=BoxOffice['Total_num_theaters']
df['Num_Screen']=BoxOffice['Total_screen_count']

In [ ]:
# OTT 매출 데이터 추출
temp_OTT_sales=OTT_sales.groupby('year')['sales'].sum()
temp_OTT_sales=temp_OTT_sales.loc[temp_OTT_sales.index.repeat(12)].reset_index(drop=True)

df['OTT_sales']=0
df['OTT_sales'].iloc[12:]=temp_OTT_sales

/tmp/ipython-input-3916500813.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['OTT_sales'].iloc[12:]=temp_OTT_sales


In [ ]:
# OTT 이용률
temp_OTT_UserRate = OTT_UserRate.sort_values(by='year')['rate']
temp_OTT_UserRate = temp_OTT_UserRate.loc[temp_OTT_UserRate.index.repeat(12)].reset_index(drop=True)
temp_OTT_UserRate.iloc[0:12]=0

df['OTT_UserRate']=temp_OTT_UserRate

In [ ]:
# Year과 Month를 기준으로 데이터프레임 병합
df=pd.merge(df, Covid, on=['Year', 'Month'], how='left')
df=pd.merge(df, GDP, on=['Year', 'Month'], how='left')
df=pd.merge(df, Unemployed, on=['Year', 'Month'], how='left')
df=pd.merge(df, N_Covid, on=['Year', 'Month'], how='left')

In [ ]:
df

,Year,Month,Num_Audience,Movie_Revenue,Num_Theaters,Num_Screen,OTT_sales,OTT_UserRate,Pandemic,GDP,unemployed,unemp_rate,youth_unemployed,youth_unemp_rate,n_covid
0,2018,1,22943456,183097961855,159,394,0,0.0,False,"476,772.5",102.0,3.7,37.1,8.7,0
1,2018,2,15551760,126413965804,125,391,0,0.0,False,"476,772.5",126.5,4.6,42.1,9.8,0
2,2018,3,12801511,103712715080,154,460,0,0.0,False,"476,772.5",125.7,4.5,50.7,11.6,0
3,2018,4,14066243,119735148093,146,439,0,0.0,False,"499,011.9",116.1,4.1,46.1,10.7,0
4,2018,5,15891715,138814866360,144,446,0,0.0,False,"499,011.9",112.1,4.0,46.0,10.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2024,8,11779803,116682706292,120,433,16019,89.2,False,"643,531.6",56.4,1.9,16.2,4.1,0
80,2024,9,10105369,100100336742,98,433,16019,89.2,False,"643,531.6",62.2,2.1,19.9,5.1,0
81,2024,10,6278029,61491564856,112,437,16019,89.2,False,"668,933.8",67.8,2.3,21.5,5.5,0
82,2024,11,6994943,66219593935,113,534,16019,89.2,False,"668,933.8",65.6,2.2,21.3,5.5,0


## **데이터 내보내기**

In [ ]:
df.to_csv('/content/drive/MyDrive/Causal_Inference/Data/merged_df.csv', index=False)